In [186]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir('/home/fractaluser/Downloads/prototype')

In [187]:
sample_data = pd.read_csv('sample_simulated_transaction_history.csv').iloc[:704778,:]

In [188]:
pd.set_option('display.max_columns', None)

In [189]:
sample_data.head()

,id,date,purchaseamount,dept,category,chain,purchasequantity,company,brand,productsize,productmeasure
0,Donor_1,2015-03-04,203.174075,Sports,Sports,site_1,1,Company1,Brand3,x,xx
1,Donor_1,2016-04-17,300.639489,Sports,Sports,site_1,1,Company1,Brand1,x,xx
2,Donor_1,2015-04-06,374.096413,Women Apparel,Dresses/Jumpsuits,site_1,1,Company2,Brand1,x,xx
3,Donor_1,2015-09-04,348.347601,Women Apparel,Dresses/Jumpsuits,site_1,1,Company2,Brand3,x,xx
4,Donor_1,2016-03-08,287.477333,Women Apparel,Dresses/Jumpsuits,site_1,1,Company1,Brand3,x,xx


In [190]:
sample_data = sample_data.drop(labels = ['productsize', 'productmeasure'], axis = 1)

In [191]:
sample_data.brand.value_counts()

Brand2    235230
Brand3    235083
Brand1    234465
Name: brand, dtype: int64

In [192]:
sample_data = sample_data.drop(labels = ['chain', 'purchasequantity'], axis = 1)

In [193]:
sample_data.head()

,id,date,purchaseamount,dept,category,company,brand
0,Donor_1,2015-03-04,203.174075,Sports,Sports,Company1,Brand3
1,Donor_1,2016-04-17,300.639489,Sports,Sports,Company1,Brand1
2,Donor_1,2015-04-06,374.096413,Women Apparel,Dresses/Jumpsuits,Company2,Brand1
3,Donor_1,2015-09-04,348.347601,Women Apparel,Dresses/Jumpsuits,Company2,Brand3
4,Donor_1,2016-03-08,287.477333,Women Apparel,Dresses/Jumpsuits,Company1,Brand3


In [194]:
sample_data.date = pd.to_datetime(sample_data.date)
sample_data = sample_data.join(pd.get_dummies(sample_data.category))
sample_data = sample_data.join(pd.get_dummies(sample_data.company))
sample_data = sample_data.join(pd.get_dummies(sample_data.brand))
sample_data = sample_data.drop(labels = ['category', 'company', 'brand'], axis = 1)

In [195]:
sample_data.head()

,id,date,purchaseamount,dept,Blazers,Dresses/Jumpsuits,Leggings,Shirts,Shoes/Sneakers,Shorts,Sports,Sweatshirts,Tees & Graphics,Tees & Polos,Tees & Tops,Toys,Company1,Company2,Brand1,Brand2,Brand3
0,Donor_1,2015-03-04,203.174075,Sports,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
1,Donor_1,2016-04-17,300.639489,Sports,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0
2,Donor_1,2015-04-06,374.096413,Women Apparel,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
3,Donor_1,2015-09-04,348.347601,Women Apparel,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
4,Donor_1,2016-03-08,287.477333,Women Apparel,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1


In [196]:
def name_trunc(x):
    x = "_".join(x.split("_", 2)[:2])
    return x

In [197]:
sample_data.id = sample_data.id.map(lambda x:name_trunc(x))

In [198]:
df_sports = sample_data[(sample_data.dept == 'Sports')].reset_index(drop = True).drop(labels = ['dept'], axis = 1).iloc[:,:3]
df_women = sample_data[(sample_data.dept == 'Women Apparel')].reset_index(drop = True).drop(labels = ['dept'], axis = 1).iloc[:,:3]
df_kidsapp = sample_data[(sample_data.dept == 'Kids Apparel')].reset_index(drop = True).drop(labels = ['dept'], axis = 1).iloc[:,:3]
df_kidsfoot = sample_data[(sample_data.dept == 'Kids Footwear')].reset_index(drop = True).drop(labels = ['dept'], axis = 1).iloc[:,:3]
df_toys = sample_data[(sample_data.dept == 'Toys')].reset_index(drop = True).drop(labels = ['dept'], axis = 1).iloc[:,:3]
df_men = sample_data[(sample_data.dept == 'Men Apparel')].reset_index(drop = True).drop(labels = ['dept'], axis = 1).iloc[:,:3]

In [199]:
sample_data.isnull().values.any()

False

In [200]:
trial = df_sports[(df_sports.id == 'Donor_1')].reset_index(drop = True)

In [201]:
trial = trial.groupby(['date']).max()

In [202]:
trial.head()

,id,purchaseamount
date,,
2015-01-02,Donor_1,458.746067
2015-01-03,Donor_1,298.217698
2015-01-05,Donor_1,422.174432
2015-01-07,Donor_1,144.846242
2015-01-08,Donor_1,468.082197


In [203]:
trial = trial.drop(labels = ['id'], axis = 1)
trial = trial.reindex(pd.date_range(sample_data.date.min(), sample_data.date.max()), fill_value = 0)

In [204]:
trial.head()

,purchaseamount
2015-01-01,0.000000
2015-01-02,458.746067
2015-01-03,298.217698
2015-01-04,0.000000
2015-01-05,422.174432


In [205]:
complete_data_sport = pd.DataFrame({'date':pd.date_range(sample_data.date.min(), sample_data.date.max())})

In [206]:
complete_data_sport.set_index('date', inplace = True)

In [207]:
for i in df_sports.id.unique():
    specific_donor = df_sports[(df_sports.id == i)].reset_index(drop = True).groupby(['date']).max().drop(labels = ['id'], axis = 1).reindex(pd.date_range(sample_data.date.min(), sample_data.date.max()), fill_value = 0)
    complete_data_sport[i] = pd.DataFrame({i:specific_donor.purchaseamount})
    

In [211]:
complete_data_sport = complete_data_sport.values

In [236]:
from keras.layers import Input, Dense
from keras.models import Model

encoding_dim = 500  
input_mat = Input(shape=(complete_data_sport.shape[1],))
encoded = Dense(encoding_dim, activation='linear')(input_mat)
decoded = Dense(complete_data_sport.shape[1], activation='linear')(encoded)

autoencoder = Model(input_mat, decoded)
encoder = Model(input_mat, encoded)

encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam', loss='mean_squared_error')

autoencoder.fit(complete_data_sport, complete_data_sport,
                epochs=100,
                batch_size=50,
                shuffle=True)

encoded_mat = encoder.predict(complete_data_sport)
decoded_mat = decoder.predict(encoded_mat)

Epoch 1/100
730/730 [==============================] - 1s 1ms/step - loss: 782.7174
Epoch 2/100
730/730 [==============================] - 0s 647us/step - loss: 419.8028
Epoch 3/100
730/730 [==============================] - 0s 648us/step - loss: 318.4942
Epoch 4/100
730/730 [==============================] - 0s 651us/step - loss: 253.3004
Epoch 5/100
730/730 [==============================] - 1s 778us/step - loss: 208.5757
Epoch 6/100
730/730 [==============================] - 0s 647us/step - loss: 177.4551
Epoch 7/100
730/730 [==============================] - 0s 637us/step - loss: 155.9020
Epoch 8/100
730/730 [==============================] - 0s 651us/step - loss: 139.0797
Epoch 9/100
730/730 [==============================] - 0s 654us/step - loss: 125.0124
Epoch 10/100
730/730 [==============================] - 0s 650us/step - loss: 113.2135
Epoch 11/100
730/730 [==============================] - 0s 640us/step - loss: 102.8392
Epoch 12/100
730/730 [==============================] 

730/730 [==============================] - 0s 656us/step - loss: 10.8377
Epoch 97/100
730/730 [==============================] - 0s 628us/step - loss: 10.6197
Epoch 98/100
730/730 [==============================] - 0s 639us/step - loss: 10.6256
Epoch 99/100
730/730 [==============================] - 0s 639us/step - loss: 10.4746
Epoch 100/100
730/730 [==============================] - 1s 741us/step - loss: 10.3776
